## Run Interative Plot to File Widget

In [15]:
import numpy as np
from test_functions1D import MultiMinimaFunc_numpy,Sinc_numpy,Sin_numpy
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from ipywidgets import FloatSlider, Button, Output, VBox, HTML

# Define the function f(x)
def f(x):
    return MultiMinimaFunc_numpy(x)

# Known global minimum of f(x) MultiMinimaFunc_numpy
f_global_min_x = -1.51035
f_global_min_value = f(f_global_min_x)

# Create values for plotting
intervaly_a, intervaly_b = -10, 120
intervalx_a, intervalx_b = -30, 30
x_values = np.linspace(intervalx_a, intervalx_b, 400)

# # Define the function f(x)
# def f(x):
#     return Sinc_numpy(x)

# # Known global minimum of f(x) sinc(x)
# f_global_min_x = 4.49341 #positive
# f_global_min_value = f(f_global_min_x)

# # Create x values for plotting
# intervaly_a, intervaly_b = -1, 2
# intervalx_a, intervalx_b = -20, 20
# x_values = np.linspace(intervalx_a, intervalx_b, 400)


# Define the function h(x)
def h(x, t, x_k):
    return f(x) + (1 / (2 * t)) * (x - x_k)**2

# Define the function u(x)
def u(x, x_hat, t):
    return f(x_hat) + (1 / (2 * t)) * (x_hat - x)**2

# Function to update the plot based on slider values
def update_plot(button):
    t = t_slider.value
    x_k = x_k_slider.value
    initial_guess = initial_guess_slider.value
    alpha = alpha_slider.value  # Get the value of alpha from the slider

    # Find the global minimum using the Nelder-Mead method
    result = minimize(h, initial_guess, args=(t, x_k), method='Nelder-Mead')

    f_values = f(x_values)
    h_values = h(x_values, t, x_k)


    if result.success:
        x_hat = result.x[0]  # Get the value of x_hat from the optimization result
        x_k_plus_1 = x_k + alpha * (x_k - x_hat)  # Compute x_{k+1}
        f_x_k_plus_1 = f(x_k_plus_1)  # Calculate f(x_{k+1})
        u_values = u(x_values, x_hat, t)

        # Plotting
        plt.figure(figsize=(12, 8))
        plt.plot(x_values, f_values, label=r'$f(x)$', color='blue')
        plt.plot(x_values, h_values, label=r'$h(x) = f(x) + \frac{1}{2t_k} (x - x_k)^2$', color='orange')

        # Plot u(x)
        plt.plot(x_values, u_values, label=r'Moreau Envelope, $u(x,t_k)$', color='green')

        # Mark the global minimum of h(x)
        global_min_value = result.fun
        plt.scatter(x_hat, global_min_value, color='red', zorder=5, label=r'Proximal, $\hat{x}_k=prox_{tf}(x_k)\approx x_k-t_k g_k$')

        # Calculate the minimum of u(x)
        u_min_value = u(x_hat, x_hat, t)
        plt.scatter(x_hat, u_min_value, color='purple', zorder=5, label='Minimum of $u(x,t_k)$', marker='x')

    # Calculate the value of f at x_k
    f_xk_value = f(x_k)
    plt.scatter(x_k, f_xk_value, color='orange', zorder=5, label=r'Current Iteration $f(x_k)$', marker='o')

    # Mark the global minimum of f(x)
    plt.scatter(f_global_min_x, f_global_min_value, color='cyan', zorder=5, label=r'Global Minima, $f(x_{true})$', marker='s')

    # Plot the point for f(x_{k+1})
    plt.scatter(x_k_plus_1, f_x_k_plus_1, color='magenta', zorder=5, 
            label=r'Next Iteration $f(x_{k+1})$, for $x_{k+1}=$' + f'{f_x_k_plus_1:.0f}\nwhere '+r'$x_{k+1}=x_k+\alpha(x_k-\hat{x}_k)$', marker='^')


    # Add titles and labels
    plt.title('Plot To Visualize Moreau Adaptive Descent')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.axhline(0, color='black', linewidth=0.5, ls='--')
    plt.axvline(0, color='black', linewidth=0.5, ls='--')

    # Set limits and grid
    plt.ylim(intervaly_a, intervaly_b)
    plt.xlim(intervalx_a, intervalx_b)
    plt.grid(which='major', linestyle='-', linewidth='0.5')
    plt.minorticks_on()
    plt.gca().xaxis.set_minor_locator(plt.MultipleLocator(1))
    plt.grid(which='minor', linestyle=':', linewidth='0.5')
    plt.legend(loc='upper right')

    # Save the figure as a PNG file
    plt.savefig("slider_plot.png", format='png', bbox_inches='tight')
    plt.close()  # Close the plot to free up memory

# Create sliders
# Create x values for plotting
x_values = np.linspace(intervalx_a, intervalx_b, 400)
t_slider = FloatSlider(value=10.0, min=1.0, max=100.0, step=1.0, description='t_k:')
x_k_slider = FloatSlider(value=20.0, min=intervalx_a, max=intervalx_b, step=1.0, description='x_k:')
initial_guess_slider = FloatSlider(value=0.0, min=intervalx_a, max=intervalx_b, step=1.0, description='Initial Guess:')
alpha_slider = FloatSlider(value=1.0, min=0.5, max=1.5, step=0.1, description=r'Step Size:')

# Create a button to trigger the update
button = Button(description="Update Plot")

# Connect the button to the update function
button.on_click(update_plot)

# Create a note explaining the sliders and the plot
note = HTML("""<h3>Instructions:</h3>
<p style="margin-bottom: 5px;">Adjust the sliders to modify the values of <strong>t</strong>, <strong>x_k</strong>, <strong>Initial Guess</strong>, and Step Size <strong>&alpha;</strong>.</p>
<p style="margin-bottom: 5px;">The <strong>Initial Guess</strong> is used with the Nelder-Mead method to compute the proximal operator <em>prox<sub>t</sub>(x<sub>k</sub>)</em>. 
   If <em>prox<sub>t</sub>(x<sub>k</sub>)</em> is not accurately found, please adjust the <strong>Initial Guess</strong> and click the "Update Plot" button to retry.</p>
<p style="margin-bottom: 5px;">The plot illustrates the functions <strong>f(x)</strong> and its Moreau envelope <strong>h(x)</strong>, allowing you to visualize the effects of your adjustments.</p>
""")

# Display sliders and button
display(VBox([t_slider, x_k_slider, initial_guess_slider, alpha_slider, button, note]))
